Compare the magnitudes produced by tractor to the magnitudes produced by the curve of growth

In [30]:
from astropy.io import fits
from astropy.table import Table
import astropy
import os
from glob import glob
import numpy as np
import fitsio


In [29]:
!pip install fitsio

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.0 MB 359 kB/s eta 0:00:01
  Created wheel for fitsio: filename=fitsio-1.1.8-cp38-cp38-linux_x86_64.whl size=1663890 sha256=e93d75106db69b9581ee2f1f5723fa6e63d138db00adc10fbee661b4d149ef65
  Stored in directory: /global/u1/m/mkwiecie/.cache/pip/wheels/05/c0/99/1ec9f7f759b5c8ac8d807f390a4032cce09c096360fa20e5e6
Successfully built fitsio


In [31]:
path = '/global/homes/m/mkwiecie/desi/sv3-clustering'
output_path = '/global/homes/m/mkwiecie/desi/legacydata/output/3962/'
base_cat = 'BGS_BRIGHT_S_clustering.dat.fits'

In [32]:
# Test code
galaxydir1 = "/global/homes/m/mkwiecie/desi/legacydata/output/3962/39627746019641337"
galaxy1 = "39627746019641337"

In [33]:

def _read_ccds_tractor_sample(prefix="custom"):
    nccds, tractor, sample = None, None, None
    refcolumn='TARGETID'

    # samplefile can exist without tractorfile when using --just-coadds
    samplefile = os.path.join(galaxydir1, "{}-sample.fits".format(galaxy1))
    print(samplefile)
    if os.path.isfile(samplefile):
        sample = astropy.table.Table(fitsio.read(samplefile, upper=True))

    tractorfile = os.path.join(
        galaxydir1, "{}-{}-tractor.fits".format(galaxy1, prefix)
    )
    print(tractorfile)
    if os.path.isfile(tractorfile):
        cols = [
            "ref_cat",
            "ref_id",
            "type",
            "sersic",
            "shape_r",
            "shape_e1",
            "shape_e2",
            "flux_g",
            "flux_r",
            "flux_z",
            "flux_ivar_g",
            "flux_ivar_r",
            "flux_ivar_z",
        ]
        print("1")
        tractor = astropy.table.Table(
            fitsio.read(tractorfile, lower=True, columns=cols)
        )  # , rows=irows
        print("2")
        # We just care about the galaxies in our sample
        wt, ws = [], []
        for ii, sid in enumerate(sample[refcolumn]):
            ww = np.where(tractor["ref_id"] == sid)[0]
            if len(ww) > 0:
                wt.append(ww)
                ws.append(ii)
        print("3")
        if len(wt) == 0:
            tractor = None
        else:
            wt = np.hstack(wt)
            ws = np.hstack(ws)
            tractor = tractor[wt]
            sample = sample[ws]
            srt = np.argsort(tractor["flux_r"])[::-1]
            tractor = tractor[srt]
            sample = sample[srt]
            assert np.all(tractor["ref_id"] == sample[refcolumn])
        print("4")
    return tractor, sample

In [34]:
def _get_mags(
    cat,
    rad="10",
    bands=["g", "r", "z"],
    kpc=False,
    pipeline=False,
    cog=False,
    R24=False,
    R25=False,
    R26=False,
):
    res = []
    for band in bands:
        mag = None
        if kpc:
            iv = cat["FLUX{}_IVAR_{}".format(rad, band.upper())][0]
            ff = cat["FLUX{}_{}".format(rad, band.upper())][0]
        elif pipeline:
            iv = cat["flux_ivar_{}".format(band.lower())]
            ff = cat["flux_{}".format(band.lower())]
        elif R24:
            mag = cat["{}_mag_sb24".format(band.lower())]
        elif R25:
            mag = cat["{}_mag_sb25".format(band.lower())]
        elif R26:
            mag = cat["{}_mag_sb26".format(band.lower())]
        elif cog:
            mag = cat["cog_mtot_{}".format(band.lower())]
        else:
            print("Thar be rocks ahead!")
        if mag:
            res.append("{:.3f}".format(mag))
        else:
            if ff > 0:
                mag = 22.5 - 2.5 * np.log10(ff)
                if iv > 0:
                    ee = 1 / np.sqrt(iv)
                    magerr = 2.5 * ee / ff / np.log(10)
                res.append("{:.3f}".format(mag))
            elif ff < 0 and iv > 0:
                # upper limit
                mag = 22.5 - 2.5 * np.log10(1 / np.sqrt(iv))
                res.append(">{:.3f}".format(mag))
            else:
                res.append("...")
    return res

In [35]:
def read_ellipsefit(galaxy, galaxydir, filesuffix='', galaxy_id='', verbose=True,
                    asTable=False):
    """Read the output of write_ellipsefit. Convert the astropy Table into a
    dictionary so we can use a bunch of legacy code.

    """
    if galaxy_id.strip() == '':
        galid = ''
    else:
        galid = '-{}'.format(galaxy_id)
    if filesuffix.strip() == '':
        fsuff = ''
    else:
        fsuff = '-{}'.format(filesuffix)

    ellipsefitfile = os.path.join(galaxydir, '{}{}-ellipse{}.fits'.format(galaxy, fsuff, galid))
        
    if os.path.isfile(ellipsefitfile):
        data = Table(fitsio.read(ellipsefitfile))

        # Optionally convert (back!) into a dictionary.
        if asTable:
            return data
        ellipsefit = {}
        for key in data.colnames:
            val = data[key][0]
            #val = data[key].tolist()[0]
            #if np.logical_not(np.isscalar(val)) and len(val) > 0:
            #    val = np.array(val, dtype=data[key].dtype)
            ellipsefit[key.lower()] = val # lowercase!
            #ellipsefit[key.lower()] = np.array(val, dtype=data[key].dtype)
    else:
        if verbose:
            print('File {} not found!'.format(ellipsefitfile))
        if asTable:
            ellipsefit = Table()
        else:
            ellipsefit = dict()

    return ellipsefit

In [36]:
tractor, sample = _read_ccds_tractor_sample(prefix="custom")

for tt, ss in zip(tractor, sample):
    g_tractor, r_tractor, z_tractor = _get_mags(tt, pipeline=True)

    galaxyid = str(tt["ref_id"])
    ellipse = read_ellipsefit(
        galaxy1,
        galaxydir1,
        filesuffix="custom",
        galaxy_id=galaxyid,
        verbose=False,
    )
    if bool(ellipse):
        g_COG, r_COG, z_COG = _get_mags(ellipse, cog=True)

    print("Tractor r mag: ", r_tractor)
    print("COG r mag: ", r_COG)

/global/homes/m/mkwiecie/desi/legacydata/output/3962/39627746019641337/39627746019641337-sample.fits
/global/homes/m/mkwiecie/desi/legacydata/output/3962/39627746019641337/39627746019641337-custom-tractor.fits
1
2
3
4
Tractor r mag:  19.493
COG r mag:  19.306


In [37]:
Correct.

SyntaxError: invalid syntax (<ipython-input-37-ca2d68100f0e>, line 1)